In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.3f}'.format
import matplotlib.pyplot as plt
import time
from datetime import datetime
import datetime
import cx_Oracle
import pyodbc
import pickle
import dateutil
from dateutil.rrule import rrule, MONTHLY, DAILY
from pyspark import SparkContext, SQLContext
from pyspark.sql import SparkSession, Row, HiveContext
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")
from plotly.subplots import make_subplots
import os.path
import math
import argparse
import pyspark.sql.functions as psf
from pyspark.sql.functions import input_file_name
import string
import random
import dash
import requests

# Estimaciones cultivo, campaña, provincia y departamento de la República Argentina

* sup_sembrada Hecteras
* sup_cosechada Hectareas
* producción Toneladas
* rendimiento kilogramo por hectárea 

https://datos.magyp.gob.ar/dataset/estimaciones-agricolas/archivo/95d066e6-8a0f-4a80-b59d-6f28f88eacd5

In [2]:
est = pd.read_csv('Estimaciones.csv',sep=";", encoding="latin1", thousands=".", decimal=",")
est.head()

,ID Provincia,Provincia,ID Departamento,Departamento,Id Cultivo,Cultivo,ID Campaña,Campana,Sup. Sembrada (Ha),Sup. Cosechada (Ha),Producción (Tn),Rendimiento (Kg/Ha)
0,6,BUENOS AIRES,854,25 DE MAYO,1,Ajo,1,1969/70,3,3,10,3333
1,6,BUENOS AIRES,854,25 DE MAYO,1,Ajo,2,1970/71,1,1,3,3000
2,6,BUENOS AIRES,14,ADOLFO GONZALES CHAVES,1,Ajo,1,1969/70,15,15,82,5467
3,6,BUENOS AIRES,14,ADOLFO GONZALES CHAVES,1,Ajo,2,1970/71,10,10,55,5500
4,6,BUENOS AIRES,14,ADOLFO GONZALES CHAVES,1,Ajo,3,1971/72,8,8,44,5500


In [3]:
est2 = est[est.Cultivo.isin(['Soja total', 'Girasol', 'Maíz', 'Trigo total'])]
est2.head()

,ID Provincia,Provincia,ID Departamento,Departamento,Id Cultivo,Cultivo,ID Campaña,Campana,Sup. Sembrada (Ha),Sup. Cosechada (Ha),Producción (Tn),Rendimiento (Kg/Ha)
49182,6,BUENOS AIRES,854,25 DE MAYO,14,Girasol,1,1969/70,30000,30000,22500,750
49183,6,BUENOS AIRES,854,25 DE MAYO,14,Girasol,2,1970/71,44000,31000,15000,484
49184,6,BUENOS AIRES,854,25 DE MAYO,14,Girasol,3,1971/72,22000,19000,10450,550
49185,6,BUENOS AIRES,854,25 DE MAYO,14,Girasol,4,1972/73,20000,5000,5100,1020
49186,6,BUENOS AIRES,854,25 DE MAYO,14,Girasol,5,1973/74,14000,14000,10800,771


In [4]:
est2.shape

(50489, 12)

In [45]:
df = est2.groupby(['Campana','Cultivo'])['Producción (Tn)'].sum().reset_index().sort_values(by='Campana', ascending=True)

df['Campana'] = df['Campana'].str[0:4]
df['Campana']  = pd.to_datetime(df['Campana'], format="%Y")

df.head()

,Campana,Cultivo,Producción (Tn)
0,1969-01-01,Girasol,1140000
1,1969-01-01,Maíz,9358372
2,1969-01-01,Soja total,26800
3,1969-01-01,Trigo total,7019726
4,1970-01-01,Girasol,830000


In [46]:
fig = px.line(df, x="Campana", y="Producción (Tn)", color='Cultivo')

fig.update_layout(title_text='Produccion (Tn)')

fig.show()

In [54]:
#Traigo los precios internacionales historicos de comtrade

API_COMTRADE = "http://comtrade.un.org/api/get"
params = {
    "freq": "A", # frecuencia anual
    "r": 32, # reporter (reporta los datos): 32 es el código de la Argentina
    "ps": "ALL", # período (any para tomar todos los años)
    "px": "S1", # sistema de clasificación de productos: SITC Rev 1 es la serie más larga
    "fmt": "JSON", # formato devuelto (CSV o JSON)
    "p": 0, # partner (contraparte comercial del reporter): 0 es el código del Mundo
    "rg": 2, # 1: importaciones, 2: exportaciones
    "cc": "2214,041,4216,044", # commodity code: 2214 "Soya" / 041 "Wheat" / 4216 "Sunflower" / 044 "Corn"
    "max": 50000, # cantidad máxima de resultados (50.000 es un máximo absoluto)
    "head": "H", # M: machine readable headers, H: human readable headers
}

res = requests.get(API_COMTRADE, params=params)
json_res = res.json()


df_comtrade = pd.DataFrame(json_res["dataset"])

columns = [
    "rgDesc", # descripción del tipo de flujo de comercio (Importación, Exportación, Re-Exportación) ("trade regime")
    "rtTitle", # nombre del país que reporta el flujo de comercio ("reporter")
    "ptTitle", # nombre del país que es contraparte del flujo de comercio reportado ("partner")
    "yr", # año
    "cmdCode", # código de la mercadería ("commodity code")
    "cmdDescE", # descripción de la mercadería ("commodity description")
    "TradeValue", # valor del flujo comercial ("trade value")
    "TradeQuantity", # cantidad física involucrada en el flujo comercial ("trade quantity")
    "qtDesc" # unidad de la cantidad utilizada ("quantity description")
]

df_comtrade["precio_ton"] = df_comtrade.TradeValue / df_comtrade.TradeQuantity * 1000

df_precios = df_comtrade.pivot(index='yr', columns='cmdDescE', values='precio_ton')
df_precios.columns = ['Maiz', 'Soja', 'Girasol','Trigo']
df_precios = df_precios.reset_index()
df_precios['yr']  = pd.to_datetime(df_precios['yr'], format="%Y")
df_precios = df_precios.rename({'yr': 'Campana'}, axis=1)

In [55]:
df_precios.head()

,Campana,Maiz,Soja,Girasol,Trigo
0,1962-01-01,41.443,NaN,206.103,61.209
1,1963-01-01,51.707,NaN,195.822,63.578
2,1964-01-01,50.337,180.000,101.099,65.298
3,1965-01-01,54.820,NaN,252.634,55.951
4,1966-01-01,53.499,75.130,240.651,55.320


In [67]:
df2 = df[df.Cultivo.isin(['Soja total'])]
df2_precios = df_precios[['Campana', 'Soja']]

merge1 = pd.merge(df2, df2_precios, on='Campana', how='left')

mask = (merge1['Campana'] > "2000-01-01")
merge1=merge1.loc[mask]

In [75]:
merge1.corr()

,Producción (Tn),Soja
Producción (Tn),1.000,0.527
Soja,0.527,1.000


In [76]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=merge1['Campana'], y=merge1['Producción (Tn)'], name="Producción (Tn)",
               mode="lines", text=merge1['Producción (Tn)'], textposition="top center"),
    secondary_y=False, )

fig.add_trace(
    go.Scatter(x=merge1['Campana'], y=merge1['Soja'], name="Precio Soja"),
    secondary_y=True, )
   
fig.show()